# Load CSV file to SQL structure

In this file, I perform the proccess of reading and loading CSV file obtained for a laboral test using Python Kernel in a venv.

In this process, the following will be carried out:

1. Connection with the **etl_workshop_first** database hosted in a **AWS RDS** server. If it doesn't exists, it will be created.

2. Creation of the database's structure, **raw_applicant** and **applicant** tables. As the proccess is running, the tables will be reset if they already exists.

3. Reading of CSV file.

4. Loading of data to **raw_applicant** table in **etl_workshop_first** database.

---
## Install requirement libraries

In [1]:
pip install -r ../code/config/requirements.txt

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'e:\UAO No Sincronizada\5° Semestre\ETL\Corte 1\Workshop001\workshop001_etl_education\venv\Scripts\python.exe -m pip install --upgrade pip' command.


---
## Proccess

Import libraries:

In [2]:
import pandas as pd
import importlib.util

spec = importlib.util.spec_from_file_location("connect_database", f"../code/connect_database.py")
connect_database = importlib.util.module_from_spec(spec)
spec.loader.exec_module(connect_database)

Information about CSV file to load:

In [3]:
location_file = '../data/candidates.csv'
raw_table_database = 'raw_applicant'
names_file = ['first_name', 'last_name', 'email', 'applicant_date', 'country', 'experience_year', 'seniority', 'technology', 'code_challenge_score', 'technical_interview_score']

Create connection:

In [4]:
# Create connection with PostgreSQL
connection = connect_database.ConnectionPostgres()
connection.make_tables()
# Read CSV file
dataframe = pd.read_csv(location_file, names = names_file, delimiter=';', header=0)
# Reorder values
dataframe.sort_values(by=['first_name', 'last_name', 'applicant_date', 'country'], inplace=True)
# Convert index to id
dataframe.reset_index(inplace=True)
dataframe.rename(columns={'index': 'id'}, inplace=True)
# Load data obtained to PostgreSQL
dataframe.to_sql(raw_table_database, connection.engine, if_exists='replace', index=False)
connection.log('Data loaded to {}: {} rows - {} columns.' .format(connection.connection_config['database'], dataframe.shape[0], dataframe.shape[1]))
# Close connection
connection.close_connection()

Data loaded:

![Raw applicant data](https://gist.github.com/assets/92474551/35a5912d-65dd-4680-aca3-d6b7e0b240f3)